In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("/Users/fernandoseguias/Desktop/ferseg/Proyectos/Programacion/iron_hack/project_1/output/cryptos_merged.csv")

#add year/month column to filter easier
df['date'] = df['date'].astype("str")
df['year'] = df['date'].apply(lambda x : x.split("/")[2])
df['month'] = df['date'].apply(lambda x : x.split("/")[1])
df['day'] = df['date'].apply(lambda x : x.split("/")[0])

#substitude zero values in market_cap column for NaN
df.loc[df["market_cap"] == 0,"market_cap"] = np.nan

#replace price (ohlc), volume and currency null values
mask_price = df["open"].isnull(), ["open", "high", "low", "close"]
df.loc[mask_price] = df["price"]

mask_vol = df["volume"].isnull(), "volume"
df.loc[mask_vol] = df["total_volume"]

mask_cur = df["currency"].isnull(), "currency"
df.loc[mask_cur] = "USD"

mask_price1 = df["price"].isnull(), "price"
df.loc[mask_price1] = df["close"]

mask_vol1 = df["total_volume"].isnull(), "total_volume"
df.loc[mask_vol1] = df["volume"]

#df filter by date (scope)
df.set_index("year", inplace=True)
scope = df.loc[["2018", "2019", "2020", "2021"], :]
scope.reset_index()

#drop duplicated columns
scope.drop(["currency", "price", "total_volume"], inplace=True, axis=1)

#supply column (market_cap = close * circulating_supply / circulating_supply = market_cap / close)
scope["circulating_supply"] = scope["market_cap"] / scope["close"]

#order columns
scope = scope.reset_index()
scope = scope[['coin', 'date', 'year', 'month', 'day', 'open', 'high', 'low', 'close', 'volume', 'market_cap', 'circulating_supply']]

#export clean csv
output_path = r"/Users/fernandoseguias/Desktop/ferseg/Proyectos/Programacion/iron_hack/project_1/output/scope.csv"
scope.to_csv(output_path, index = False)

#dictionary creation dict = {coin type(str) : data per coin type(df)}
tickers = scope["coin"].unique()

dict_tickers = {}

for key in tickers.tolist():
    dict_tickers[key] = key
    dict_tickers[key] = scope.loc[scope.coin == key]

output_path = r"/Users/fernandoseguias/Desktop/ferseg/Proyectos/Programacion/iron_hack/project_1/output/dict_scope.json"
scope.to_json(output_path, index = True)

dict_tickers["ftm"] #example value for key ftm (fantom)

,coin,date,year,month,day,open,high,low,close,volume,market_cap,circulating_supply
3244,ftm,30/10/2018,2018,10,30,0.01398,0.01398,0.01398,0.01398,6.941221e+06,NaN,NaN
3245,ftm,31/10/2018,2018,10,31,0.02102,0.02102,0.02102,0.02102,7.681527e+06,2.951860e+07,1.404310e+09
3246,ftm,01/11/2018,2018,11,01,0.02454,0.02454,0.02454,0.02454,1.198239e+07,3.420793e+07,1.393966e+09
3247,ftm,02/11/2018,2018,11,02,0.02250,0.02250,0.02250,0.02250,7.906760e+06,3.152514e+07,1.401117e+09
3248,ftm,03/11/2018,2018,11,03,0.02218,0.02218,0.02218,0.02218,4.810542e+06,3.091862e+07,1.393987e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
18690,ftm,30/12/2021,2021,12,30,2.13240,2.22500,2.04020,2.10250,2.280050e+05,5.366687e+09,2.552527e+09
18691,ftm,31/12/2021,2021,12,31,2.10250,2.33510,2.08360,2.25140,1.927540e+05,NaN,NaN
21362,ftm,27/07/2021,2021,07,27,0.20956,0.20956,0.20956,0.20956,7.798620e+07,5.388942e+08,2.571551e+09
21363,ftm,10/09/2021,2021,09,10,1.65739,1.65739,1.65739,1.65739,1.817521e+09,4.214438e+09,2.542816e+09
